In [ ]:
import requests
import json

# Define the URL for the API
url = "http://localhost:8000/v1/chat/completions"

# Set up the headers
headers = {
    "Content-Type": "application/json"
}

# Set up the data for the request
data = {
    "model": "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    "messages": [
        {
            "role": "user",
            "content": "How to stop procastinating"
        }
    ]
}

# Send the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check if the request was successful
if response.status_code == 200:
    # Parse the response JSON
    response_json = response.json()
    # Print the model's reply
    print("Model's response:", response_json['choices'][0]['message']['content'])
else:
    print(f"Failed to get a response. Status code: {response.status_code}")


Creation of database

In [6]:
import sqlite3
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Function to generate a single row of data
def generate_row():
    return {
        'id': fake.uuid4(),
        'name': fake.name(),
        'email': fake.email(),
        'address': fake.address(),
        'phone_number': fake.phone_number(),
        'job': fake.job(),
        'company': fake.company(),
        'date_of_birth': fake.date_of_birth().strftime('%Y-%m-%d'),  # Format for SQL
        'ssn': fake.ssn(),
        'credit_card_number': fake.credit_card_number(),
        'credit_card_expire': fake.credit_card_expire(),
        'credit_card_provider': fake.credit_card_provider(),
        'bank_country': fake.country(),
        'currency_code': fake.currency_code(),
        'amount': round(random.uniform(100, 10000), 2),  # Round to 2 decimal places
        'transaction_date': fake.date_time_this_decade().strftime('%Y-%m-%d %H:%M:%S'),  # Format for SQL
    }

# Create and connect to the large SQL database
large_db = sqlite3.connect('large_database.db')
large_cursor = large_db.cursor()

# Create a table in the large database
large_cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    id TEXT PRIMARY KEY,
    name TEXT,
    email TEXT,
    address TEXT,
    phone_number TEXT,
    job TEXT,
    company TEXT,
    date_of_birth TEXT,
    ssn TEXT,
    credit_card_number TEXT,
    credit_card_expire TEXT,
    credit_card_provider TEXT,
    bank_country TEXT,
    currency_code TEXT,
    amount REAL,
    transaction_date TEXT
)
''')

# Generate and insert 200,000 rows into the large database
for _ in range(200000):
    row = generate_row()
    large_cursor.execute('''
    INSERT INTO customers (
        id, name, email, address, phone_number, job, company, date_of_birth, ssn,
        credit_card_number, credit_card_expire, credit_card_provider, bank_country,
        currency_code, amount, transaction_date
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', tuple(row.values()))

# Commit changes and close the large database
large_db.commit()
large_db.close()

# Create and connect to the small SQL database
small_db = sqlite3.connect('small_database.db')
small_cursor = small_db.cursor()

# Create a table in the small database
small_cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    id TEXT PRIMARY KEY,
    name TEXT,
    email TEXT,
    address TEXT,
    phone_number TEXT,
    job TEXT,
    company TEXT,
    date_of_birth TEXT,
    ssn TEXT,
    credit_card_number TEXT,
    credit_card_expire TEXT,
    credit_card_provider TEXT,
    bank_country TEXT,
    currency_code TEXT,
    amount REAL,
    transaction_date TEXT
)
''')

# Connect to the large database again to fetch a subset of data
large_db = sqlite3.connect('large_database.db')
large_cursor = large_db.cursor()

# Fetch 2,000 random rows from the large database
large_cursor.execute('SELECT * FROM customers ORDER BY RANDOM() LIMIT 2000')
rows = large_cursor.fetchall()

# Insert the subset into the small database
for row in rows:
    small_cursor.execute('''
    INSERT INTO customers (
        id, name, email, address, phone_number, job, company, date_of_birth, ssn,
        credit_card_number, credit_card_expire, credit_card_provider, bank_country,
        currency_code, amount, transaction_date
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', row)

# Commit changes and close the databases
small_db.commit()
small_db.close()
large_db.close()

print("Large SQL database saved as 'large_database.db'")
print("Small SQL database saved as 'small_database.db'")

Large SQL database saved as 'large_database.db'
Small SQL database saved as 'small_database.db'


English to sql

In [ ]:
import requests
import json
import sqlite3
import re

# Define the URL for the API
url = "http://localhost:8000/v1/chat/completions"

# Set up the headers
headers = {
    "Content-Type": "application/json"
}

def ask_model(prompt):
    data = {
        "model": "defog/llama-3-sqlcoder-8b",  # Updated model name
        "temperature": 0.0001,  # Lower value for more deterministic responses
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        model_output = response.json()["choices"][0]["message"]["content"]
        
        # ✅ Print raw output to see what the model is returning
        print("\n🔍 Model Raw Output:\n", model_output)  
        
        return model_output
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

# Function to convert natural language to SQL
def natural_language_to_sql(user_command):
    prompt = f"""
    ### Task:
    Convert the following natural language command into a correctly formatted SQLite SQL query.

    ### Database Schema:
    Table: customers
    Columns:
      - id (TEXT, PRIMARY KEY)
      - name (TEXT)
      - email (TEXT)
      - address (TEXT)
      - phone_number (TEXT)
      - job (TEXT)
      - company (TEXT)
      - date_of_birth (TEXT)
      - ssn (TEXT)
      - credit_card_number (TEXT)
      - credit_card_expire (TEXT)
      - credit_card_provider (TEXT)
      - bank_country (TEXT)
      - currency_code (TEXT)
      - amount (REAL)
      - transaction_date (TEXT)

    ### Examples:
    **User Input:** What is the total number of customers?
    **SQL Output:** SELECT COUNT(*) FROM customers;

    **User Input:** What is the total value of all transactions?
    **SQL Output:** SELECT SUM(amount) FROM customers;

    **User Input:** Show all customers born after 1990 in the USA.
    **SQL Output:** SELECT name FROM customers WHERE date_of_birth > '1990-01-01' AND bank_country = 'USA';

    ### User Input:
    {user_command}

    ### SQL Output:
    """

    response = ask_model(prompt)

    # ✅ Stricter regex to force correct SQL extraction
    sql_query_match = re.search(r"(?i)SELECT\s+.*?\s+FROM\s+customers\s*;", response, re.DOTALL)

    if sql_query_match:
        sql_query = sql_query_match.group(0).strip()
    else:
        print("⚠️ Warning: Model response did not return a valid SQL query. Retrying with fallback.")

        # ✅ Dynamically generate fallback query based on input
        if "total value of all transactions" in user_command.lower():
            sql_query = "SELECT SUM(amount) FROM customers;"
        elif "total number of customers" in user_command.lower():
            sql_query = "SELECT COUNT(*) FROM customers;"
        else:
            sql_query = "SELECT * FROM customers LIMIT 5;"  # Safe fallback

    return sql_query.strip()

# Function to execute SQL on the database
def execute_sql(database_path, sql_query):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()

        print(f"\n🔍 Executing SQL Query: {sql_query}\n")

        # Execute the SQL query
        cursor.execute(sql_query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]  # Get column names

        conn.close()

        if not results:
            print("⚠️ No results found. Try refining your query.")
            return "No matching records found."

        # ✅ Format output correctly
        if len(columns) == 1:  # If only one column, flatten output
            return [row[0] for row in results]
        else:
            return results  # Return normal table output

    except sqlite3.Error as e:
        print(f"⚠️ SQL Error: {e}")
        return f"An error occurred: {e}"
    except Exception as e:
        print(f"⚠️ Unexpected Error: {e}")
        return f"An unexpected error occurred: {e}"

# Main function to interact with the user
def main():
    database_path = "large_database.db"  # Path to SQLite database

    while True:
        user_command = input("Enter your query (or type 'exit' to quit): ")
        if user_command.lower() == "exit":
            break

        sql_query = natural_language_to_sql(user_command)
        print(f"\n📝 Generated SQL Query:\n{sql_query}")

        results = execute_sql(database_path, sql_query)

        print("\n📌 Query Results:\n")
        if isinstance(results, str):
            print(results)
        elif isinstance(results, list) and all(isinstance(i, (int, float, str)) for i in results):
            # ✅ If output is a single-column list, print in a readable format
            print(", ".join(map(str, results)))
        else:
            for row in results:
                print(row)

if __name__ == "__main__":
    main()

Enter your query (or type 'exit' to quit):  Total number of customers on the database



🔍 Model Raw Output:
 SELECT COUNT(*) FROM customers;

📝 Generated SQL Query:
SELECT COUNT(*) FROM customers;

🔍 Executing SQL Query: SELECT COUNT(*) FROM customers;


📌 Query Results:

200000


Enter your query (or type 'exit' to quit):  Find all customers from the USA.



🔍 Model Raw Output:
 SELECT name FROM customers WHERE bank_country = 'USA';
⚠️ Warning: Model response did not return a valid SQL query. Retrying with fallback.

📝 Generated SQL Query:
SELECT * FROM customers LIMIT 5;

🔍 Executing SQL Query: SELECT * FROM customers LIMIT 5;


📌 Query Results:

('06e68b4d-28b3-4821-9e66-2a4436dcac5f', 'Eric Brown', 'narmstrong@example.com', '1837 Michael Curve Apt. 481\nNew Jeffreyview, NE 25505', '001-802-974-0847x1012', 'Embryologist, clinical', 'Griffin, Carlson and Hunter', '1928-03-31', '546-70-8535', '4753671023330532', '02/32', 'VISA 16 digit', 'Netherlands', 'DJF', 4353.64, '2021-01-08 15:30:02')
('262b52f3-c61d-48df-b14d-fba2d06a412f', 'Kimberly Miller', 'suesanchez@example.net', '507 Rebecca Garden\nPort Noah, MS 06187', '001-312-785-7333x118', 'TEFL teacher', 'Young-Johnson', '1930-01-16', '272-32-8968', '6512125080111589', '03/30', 'JCB 16 digit', 'Saint Barthelemy', 'CNY', 7149.81, '2022-01-08 04:52:30')
('cab9687b-d4df-4c52-8579-da49995bd

Without regex


In [8]:
import requests
import json
import sqlite3
import time

# Define the URL for the API
url = "http://localhost:8000/v1/chat/completions"
time_details = []  # Change to a list to store multiple entries

# Set up the headers
headers = {
    "Content-Type": "application/json"
}

def ask_model(prompt):
    data = {
        "model": "defog/llama-3-sqlcoder-8b",  # Updated model name
        "temperature": 0.0001,  # Lower value for more deterministic responses
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        model_output = response.json()["choices"][0]["message"]["content"]
        
        # ✅ Print raw output to see what the model is returning
        print("\n🔍 Model Raw Output:\n", model_output)  
        
        return model_output
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

# Function to convert natural language to SQL
def natural_language_to_sql(user_command):
    prompt = f"""
    ### Task:
Convert the following natural language command into a correctly formatted SQLite SQL query.

### Important Notes:
1. Use only SQLite-compatible functions. SQLite does NOT support the following functions:
   - `EXTRACT`
   - `AGE`
   - `TO_DATE`
   - `DATE_PART`
   - `DATE_TRUNC`
2. For date calculations, use SQLite's `julianday` function.
3. Return ONLY the SQL query. NO explanations, NO markdown (`sql ... `), NO descriptions.
4. The output must be a direct SQL query, formatted correctly for SQLite.
5. Do NOT include anything else.

### Database Schema:
Table: customers
Columns:
  - id (TEXT, PRIMARY KEY)
  - name (TEXT)
  - email (TEXT)
  - address (TEXT)
  - phone_number (TEXT)
  - job (TEXT)
  - company (TEXT)
  - date_of_birth (TEXT)
  - ssn (TEXT)
  - credit_card_number (TEXT)
  - credit_card_expire (TEXT)
  - credit_card_provider (TEXT)
  - bank_country (TEXT)
  - currency_code (TEXT)
  - amount (REAL)
  - transaction_date (TEXT)

### Examples:
**User Input:** What is the total number of customers?
**SQL Output:** SELECT COUNT(*) FROM customers;

**User Input:** What is the total value of all transactions?
**SQL Output:** SELECT SUM(amount) FROM customers;

**User Input:** Show all customers born after 1990 in the USA.
**SQL Output:** SELECT name FROM customers WHERE date_of_birth > '1990-01-01' AND bank_country = 'USA';

**User Input:** What is the average age of customers?
**SQL Output:** SELECT AVG((julianday('now') - julianday(date_of_birth)) / 365.25) AS average_age FROM customers;

### User Input:
{user_command}

### SQL Output:
    """

    response = ask_model(prompt)

    # ✅ Basic validation: Check if the response starts with SELECT and ends with a semicolon
    if response.strip().upper().startswith("SELECT") and response.strip().endswith(";"):
        sql_query = response.strip()
    else:
        print("⚠️ Warning: Model response did not return a valid SQL query. Retrying with fallback.")
        sql_query = "SELECT * FROM customers LIMIT 5;"  # Safe fallback

    return sql_query

# Function to execute SQL on the database
def execute_sql(database_path, sql_query):
    global time_details
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()

        print(f"\n🔍 Executing SQL Query: {sql_query}\n")

        # ✅ Start timing
        start_time = time.time()

        # Execute the SQL query
        cursor.execute(sql_query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]  # Get column names

        # ✅ End timing
        end_time = time.time()
        execution_time = end_time - start_time

        # Append the query and execution time to the list
        time_details.append({
            "Query": sql_query,
            "Execution Time": execution_time
        })

        conn.close()

        if not results:
            return "No matching records found.", execution_time

        # ✅ Format output correctly
        if len(columns) == 1:  # If only one column, flatten output
            return [row[0] for row in results], execution_time
        else:
            return results, execution_time  # Return normal table output and execution time

    except sqlite3.Error as e:
        print(f"⚠️ SQL Error: {e}")

        # ✅ Fallback for invalid SQL
        if "average age" in sql_query.lower():
            print("🔄 Attempting fallback query for average age calculation.")
            fallback_query = "SELECT AVG((julianday('now') - julianday(date_of_birth)) / 365.25) AS average_age FROM customers;"
            return execute_sql(database_path, fallback_query)
        else:
            return f"An error occurred: {e}", 0
    except Exception as e:
        print(f"⚠️ Unexpected Error: {e}")
        return f"An unexpected error occurred: {e}", 0

# Main function to interact with the user
def main():
    database_path = "large_database.db"  # Path to SQLite database

    while True:
        user_command = input("Enter your query (or type 'exit' to quit): ")
        if user_command.lower() == "exit":
            break

        sql_query = natural_language_to_sql(user_command)
        print(f"\n📝 Generated SQL Query:\n{sql_query}")

        results, execution_time = execute_sql(database_path, sql_query)

        print("\n📌 Query Results:\n")
        if isinstance(results, str):
            print(results)
        elif isinstance(results, list) and all(isinstance(i, (int, float, str)) for i in results):
            # ✅ If output is a single-column list, print in a readable format
            print(", ".join(map(str, results)))
        else:
            for row in results:
                print(row)

        # ✅ Print execution time
        print(f"\n⏱️ Execution Time: {execution_time:.4f} seconds")

    # Print all stored time details
    print("\n📊 All Query Execution Times:")
    for detail in time_details:
        print(f"Query: {detail['Query']}\nExecution Time: {detail['Execution Time']:.4f} seconds\n")

if __name__ == "__main__":
    main()

Enter your query (or type 'exit' to quit):  What is the total number of customers?



🔍 Model Raw Output:
 SELECT COUNT(*) FROM customers;

📝 Generated SQL Query:
SELECT COUNT(*) FROM customers;

🔍 Executing SQL Query: SELECT COUNT(*) FROM customers;


📌 Query Results:

200000

⏱️ Execution Time: 0.5436 seconds


Enter your query (or type 'exit' to quit):  What is the sum of all transaction amounts?



🔍 Model Raw Output:
 SELECT SUM(amount) FROM customers;

📝 Generated SQL Query:
SELECT SUM(amount) FROM customers;

🔍 Executing SQL Query: SELECT SUM(amount) FROM customers;


📌 Query Results:

1008833339.6899996

⏱️ Execution Time: 0.3971 seconds


Enter your query (or type 'exit' to quit):  Most common customer name 



🔍 Model Raw Output:
 SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;

📝 Generated SQL Query:
SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;

🔍 Executing SQL Query: SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;


📌 Query Results:

('Michael Smith', 81)

⏱️ Execution Time: 0.6437 seconds


Enter your query (or type 'exit' to quit):  exit



📊 All Query Execution Times:
Query: SELECT COUNT(*) FROM customers;
Execution Time: 0.5436 seconds

Query: SELECT SUM(amount) FROM customers;
Execution Time: 0.3971 seconds

Query: SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;
Execution Time: 0.6437 seconds



In [6]:
for detail in time_details:
        print(f"Query: {detail['Query']}\nExecution Time: {detail['Execution Time']:.4f} seconds")


Query: SELECT COUNT(*) FROM customers;
Execution Time: 0.4019 seconds
Query: SELECT SUM(amount) FROM customers;
Execution Time: 0.4310 seconds
Query: SELECT AVG(amount) FROM customers;
Execution Time: 0.3585 seconds
Query: SELECT bank_country, SUM(amount) AS total_transaction_amount FROM customers GROUP BY bank_country;
Execution Time: 0.5439 seconds


Small Database

In [9]:
import requests
import json
import sqlite3
import time

# Define the URL for the API
url = "http://localhost:8000/v1/chat/completions"
time_detailsSM1 = []  # List to store multiple entries

# Set up the headers for API requests
headers = {
    "Content-Type": "application/json"
}

def ask_model(prompt):
    data = {
        "model": "defog/llama-3-sqlcoder-8b",  # Updated model name
        "temperature": 0.0001,  # Lower value for more deterministic responses
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))

    if response.status_code == 200:
        model_output = response.json()["choices"][0]["message"]["content"]
        print("\n🔍 Model Raw Output:\n", model_output)
        return model_output
    else:
        raise Exception(f"API request failed with status code {response.status_code}")

def natural_language_to_sql(user_command):
    prompt = f"""
    ### Task:
Convert the following natural language command into a correctly formatted SQLite SQL query.

### Important Notes:
1. Use only SQLite-compatible functions. SQLite does NOT support the following functions:
   - `EXTRACT`
   - `AGE`
   - `TO_DATE`
   - `DATE_PART`
   - `DATE_TRUNC`
2. For date calculations, use SQLite's `julianday` function.
3. Return ONLY the SQL query. NO explanations, NO markdown (`sql ... `), NO descriptions.
4. The output must be a direct SQL query, formatted correctly for SQLite.
5. Do NOT include anything else.

### Database Schema:
Table: customers
Columns:
  - id (TEXT, PRIMARY KEY)
  - name (TEXT)
  - email (TEXT)
  - address (TEXT)
  - phone_number (TEXT)
  - job (TEXT)
  - company (TEXT)
  - date_of_birth (TEXT)
  - ssn (TEXT)
  - credit_card_number (TEXT)
  - credit_card_expire (TEXT)
  - credit_card_provider (TEXT)
  - bank_country (TEXT)
  - currency_code (TEXT)
  - amount (REAL)
  - transaction_date (TEXT)

### Examples:
**User Input:** What is the total number of customers?
**SQL Output:** SELECT COUNT(*) FROM customers;

**User Input:** What is the total value of all transactions?
**SQL Output:** SELECT SUM(amount) FROM customers;

**User Input:** Show all customers born after 1990 in the USA.
**SQL Output:** SELECT name FROM customers WHERE date_of_birth > '1990-01-01' AND bank_country = 'USA';

**User Input:** What is the average age of customers?
**SQL Output:** SELECT AVG((julianday('now') - julianday(date_of_birth)) / 365.25) AS average_age FROM customers;

### User Input:
{user_command}

### SQL Output:
    """
    response = ask_model(prompt)

    if response.strip().upper().startswith("SELECT") and response.strip().endswith(";"):
        sql_query = response.strip()
    else:
        print("⚠️ Warning: Model response did not return a valid SQL query. Retrying with fallback.")
        sql_query = "SELECT * FROM customers LIMIT 5;"  # Safe fallback

    return sql_query

def compute_scaling_factor(small_db_path, large_db_path):
    """
    Automatically computes the scaling factor as the ratio of the number of rows in the large database 
    to the number of rows in the small database.
    """
    try:
        # Count rows in small database
        small_conn = sqlite3.connect(small_db_path)
        small_cursor = small_conn.cursor()
        small_cursor.execute("SELECT COUNT(*) FROM customers")
        small_count = small_cursor.fetchone()[0]
        small_conn.close()

        # Count rows in large database
        large_conn = sqlite3.connect(large_db_path)
        large_cursor = large_conn.cursor()
        large_cursor.execute("SELECT COUNT(*) FROM customers")
        large_count = large_cursor.fetchone()[0]
        large_conn.close()

        if small_count > 0:
            return large_count / small_count
        else:
            return 1
    except Exception as e:
        print(f"Error computing scaling factor: {e}")
        return 1

def execute_sql(database_path, sql_query, scaling_factor=1):
    """
    Execute the given SQL query and apply a scaling factor to aggregate results 
    (e.g. SUM or COUNT) if necessary.
    """
    global time_details
    try:
        conn = sqlite3.connect(database_path)
        cursor = conn.cursor()

        print(f"\n🔍 Executing SQL Query: {sql_query}\n")
        
        # Start timing
        start_time = time.time()
        cursor.execute(sql_query)
        results = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        end_time = time.time()
        execution_time = end_time - start_time
        
        # Log the query execution details
        time_detailsSM1.append({
            "Query": sql_query,
            "Execution Time": execution_time
        })
        
        # If the scaling factor is greater than 1 and the query uses aggregate functions,
        # scale the result appropriately.
        if scaling_factor > 1:
            query_upper = sql_query.upper()
            # Avoid scaling AVG since averages should not be multiplied
            if ("SUM(" in query_upper or "COUNT(" in query_upper) and "AVG(" not in query_upper:
                # For a single aggregate value (one row, one column)
                if len(results) == 1 and len(results[0]) == 1 and isinstance(results[0][0], (int, float)):
                    results = [[results[0][0] * scaling_factor]]
                else:
                    # For grouped results, assume that the aggregate is in the second column
                    new_results = []
                    for row in results:
                        row_list = list(row)
                        if len(row_list) >= 2 and isinstance(row_list[1], (int, float)):
                            row_list[1] *= scaling_factor
                        elif len(row_list) == 1 and isinstance(row_list[0], (int, float)):
                            row_list[0] *= scaling_factor
                        new_results.append(tuple(row_list))
                    results = new_results

        conn.close()

        if not results:
            return "No matching records found.", execution_time

        if len(columns) == 1:  # Flatten output for single-column results
            return [row[0] for row in results], execution_time
        else:
            return results, execution_time

    except sqlite3.Error as e:
        print(f"⚠️ SQL Error: {e}")
        if "average age" in sql_query.lower():
            print("🔄 Attempting fallback query for average age calculation.")
            fallback_query = "SELECT AVG((julianday('now') - julianday(date_of_birth)) / 365.25) AS average_age FROM customers;"
            return execute_sql(database_path, fallback_query, scaling_factor)
        else:
            return f"An error occurred: {e}", 0
    except Exception as e:
        print(f"⚠️ Unexpected Error: {e}")
        return f"An unexpected error occurred: {e}", 0

def main():
    # Define database paths
    small_db_path = "small_database.db"
    large_db_path = "large_database.db"
    
    # Automatically compute the scaling factor using the row counts
    scaling_factor = compute_scaling_factor(small_db_path, large_db_path)
    print(f"Computed Scaling Factor: {scaling_factor}")
    
    # Use the small database for query testing
    database_path = small_db_path

    while True:
        user_command = input("Enter your query (or type 'exit' to quit): ")
        if user_command.lower() == "exit":
            break

        sql_query = natural_language_to_sql(user_command)
        print(f"\n📝 Generated SQL Query:\n{sql_query}")

        results, execution_time = execute_sql(database_path, sql_query, scaling_factor=scaling_factor)

        print("\n📌 Query Results:\n")
        if isinstance(results, str):
            print(results)
        elif isinstance(results, list) and all(isinstance(i, (int, float, str)) for i in results):
            print(", ".join(map(str, results)))
        else:
            for row in results:
                print(row)

        print(f"\n⏱️ Execution Time: {execution_time:.4f} seconds")

    print("\n📊 All Query Execution Times:")
    for detail in time_detailsSM1:
        print(f"Query: {detail['Query']}\nExecution Time: {detail['Execution Time']:.4f} seconds\n")

if __name__ == "__main__":
    main()


Computed Scaling Factor: 100.0


Enter your query (or type 'exit' to quit):  What is the total number of customers?



🔍 Model Raw Output:
 SELECT COUNT(*) FROM customers;

📝 Generated SQL Query:
SELECT COUNT(*) FROM customers;

🔍 Executing SQL Query: SELECT COUNT(*) FROM customers;


📌 Query Results:

200000.0

⏱️ Execution Time: 0.0073 seconds


Enter your query (or type 'exit' to quit):  What is the sum of all transaction amounts?



🔍 Model Raw Output:
 SELECT SUM(amount) FROM customers;

📝 Generated SQL Query:
SELECT SUM(amount) FROM customers;

🔍 Executing SQL Query: SELECT SUM(amount) FROM customers;


📌 Query Results:

1003485565.0

⏱️ Execution Time: 0.0108 seconds


Enter your query (or type 'exit' to quit):  Most common customer name



🔍 Model Raw Output:
 SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;

📝 Generated SQL Query:
SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;

🔍 Executing SQL Query: SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;


📌 Query Results:

('Mark Smith', 400.0)

⏱️ Execution Time: 0.0228 seconds


Enter your query (or type 'exit' to quit):  exit



📊 All Query Execution Times:
Query: SELECT COUNT(*) FROM customers;
Execution Time: 0.0073 seconds

Query: SELECT SUM(amount) FROM customers;
Execution Time: 0.0108 seconds

Query: SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;
Execution Time: 0.0228 seconds



In [10]:
for detail in time_detailsSM1:
        print(f"Query: {detail['Query']}\nExecution Time: {detail['Execution Time']:.4f} seconds")


Query: SELECT COUNT(*) FROM customers;
Execution Time: 0.0073 seconds
Query: SELECT SUM(amount) FROM customers;
Execution Time: 0.0108 seconds
Query: SELECT name, COUNT(*) AS frequency FROM customers GROUP BY name ORDER BY frequency DESC LIMIT 1;
Execution Time: 0.0228 seconds
